# Churn Predictive Analytics using Amazon SageMaker and Snowflake

---
## Background

The purpose of this lab is to demonstrate the basics of building an advanced analytics solution using Amazon SageMaker on data stored in Snowflake. In this notebook we will create a customer churn analytics solution by training an XGBoost churn model, and batching churn prediction scores into a data warehouse. 

(Need to update) This notebook extends one of the example tutorial notebooks: [Customer Churn Prediction with XGBoost](https://github.com/awslabs/amazon-sagemaker-examples/blob/master/introduction_to_applying_machine_learning/xgboost_customer_churn/xgboost_customer_churn_neo.ipynb). The extended learning objectives are highlighted in bold below.

#### Learning Objectives 

 - **Learn how to query ground truth data from our data warehouse into a pandas dataframe for exploration and feature engineering.**
 - Train an XGBoost model to perform churn prediction.
 - **Learn how to run a Batch Transform job to calculate churn scores in batch.**
 - Optimize your model using SageMaker Neo.
 - Upload the Churn Score results back to Snowflake to perform basic analysis. 

---

## Prerequisites


In summary:
 - You've built the lab environment using this CloudFormation [template](https://snowflake-corp-se-workshop.s3-us-west-1.amazonaws.com/sagemaker-snowflake-workshop-v1/sagemaker/snowflake-sagemaker-notebook-v1.1.yaml). This template installs the Snowflake python connector within your Jupyter instance.
 - You've taken note of the Snowflake credentials in the lab guide.
 - This notebook should be running in your default VPC. 
 - Snowflake traffic uses port 443.
 
---

## Setup



Run the cell below to import Python libraries required by this notebook.

The IAM role arn used to give training and hosting access to your data. By default, we'll use the IAM permissions that have been allocated to your notebook instance. The role should have the permissions to access your S3 bucket, and full execution permissions on Amazon SageMaker. In practice, you could minimize the scope of requried permissions.

Now let's set the S3 bucket and prefix that you want to use for training and model data. This bucket should be created within the same region as the Notebook Instance, training, and hosting. 

- Replace <<'REPLACE WITH YOUR BUCKET NAME'>> with the name of your bucket.

---
## Data

Mobile operators have historical records on which customers ultimately ended up churning and which continued using the service. We can use this historical information to construct an ML model of one mobile operator’s churn using a process called training. After training the model, we can pass the profile information of an arbitrary customer (the same profile information that we used to train the model) to the model, and have the model predict whether this customer is going to churn. Of course, we expect the model to make mistakes–after all, predicting the future is tricky business! But I’ll also show how to deal with prediction errors.

The dataset we use is publicly available and was mentioned in the book [Discovering Knowledge in Data](https://www.amazon.com/dp/0470908742/) by Daniel T. Larose. It is attributed by the author to the University of California Irvine Repository of Machine Learning Datasets.  In the previous steps, this dataset was loaded into the CUSTOMER_CHURN table in your Snowflake instance.

Provide the connection and credentials required to connect to your Snowflake account. You'll need to modify the cell below with the appropriate **ACCOUNT** for your Snowflake trial. If you followed the lab guide instructions, the username and password below will work. 

**NOTE:** For Snowflake accounts in regions other than US WEST add the Region ID after a period <ACCOUNT>.<REGION ID> i.e. XYZ123456.US-EAST-1. 

In practice, security standards might prohibit you from providing credentials in clear text. As a best practice in production, you should utilize a service like [AWS Secrets Manager](https://docs.aws.amazon.com/secretsmanager/latest/userguide/intro.html) to manage your database credentials.

In [1]:
import snowflake.connector # tested on snowflake-connector-python==2.3.1 
# Connecting to Snowflake using the default authenticator
ctx = snowflake.connector.connect(
  user='HOPSWORKS',
  password='HOPSWORKS123',
  account='ra96958.eu-central-1',
  warehouse='HOPSWORKS_WH',
  database='ML_WORKSHOP',
  schema='PUBLIC'
)

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log
14,application_1599768044375_0017,pyspark,idle,Link,Link


SparkSession available as 'spark'.
/srv/hops/anaconda/envs/theenv/lib/python3.6/site-packages/snowflake/connector/options.py:40: UserWarning: You have an incompatible version of 'pyarrow' installed, please install a version that adheres to: 'pyarrow<0.18.0,>=0.17.0; extra == "pandas"'
  warn_incompatible_dep('pyarrow', _installed_pyarrow.version, _expected_version)

---
## Explore

Now we can run queries against your database. 

However, in practice, the data table will often contain more data than what is practical to operate on within a notebook instance, or relevant attributes are spread across multiple tables. Being able to run SQL queries and loading the data into a pandas dataframe will be helpful during the initial stages of development. Check out the Spark integration for a fully scalable solution. [Snowflake Connector for Spark](https://docs.snowflake.net/manuals/user-guide/spark-connector.html)


In [2]:
import pandas as pd
# Query Snowflake Data
cs=ctx.cursor()
allrows=cs.execute("""select Cust_ID,STATE,ACCOUNT_LENGTH,AREA_CODE,PHONE,INTL_PLAN,VMAIL_PLAN,VMAIL_MESSAGE,
                   DAY_MINS,DAY_CALLS,DAY_CHARGE,EVE_MINS,EVE_CALLS,EVE_CHARGE,NIGHT_MINS,NIGHT_CALLS,
                   NIGHT_CHARGE,INTL_MINS,INTL_CALLS,INTL_CHARGE,CUSTSERV_CALLS,
                   CHURN from CUSTOMER_CHURN """).fetchall()

churn = pd.DataFrame(allrows)
churn.columns=['Cust_id','State','Account Length','Area Code','Phone','Intl Plan', 'VMail Plan', 'VMail Message','Day Mins',
            'Day Calls', 'Day Charge', 'Eve Mins', 'Eve Calls', 'Eve Charge', 'Night Mins', 'Night Calls','Night Charge',
            'Intl Mins','Intl Calls','Intl Charge','CustServ Calls', 'Churn?']

pd.set_option('display.max_columns', 500)     # Make sure we can see all of the columns
pd.set_option('display.max_rows', 10)         # Keep the output on one page
churn

      Cust_id State  Account Length  Area Code     Phone Intl Plan VMail Plan  \
0           1    KS             128        415  382-4657        no        yes   
1           2    OH             107        415  371-7191        no        yes   
2           3    NJ             137        415  358-1921        no         no   
3           4    OH              84        408  375-9999       yes         no   
4           5    MA             121        510  355-9993        no        yes   
...       ...   ...             ...        ...       ...       ...        ...   
3328     3329    OK             138        510  406-5532        no        yes   
3329     3330    AL              89        510  347-2016        no         no   
3330     3331    ND             131        408  393-9548        no        yes   
3331     3332    ND             122        408  395-1901        no         no   
3332     3333    OH             136        408  392-1547       yes         no   

      VMail Message  Day Mi

By modern standards, it’s a relatively small dataset, with only 3,333 records, where each record uses 21 attributes to describe the profile of a customer of an unknown US mobile operator. The attributes are:

- `State`: the US state in which the customer resides, indicated by a two-letter abbreviation; for example, OH or NJ
- `Account Length`: the number of days that this account has been active
- `Area Code`: the three-digit area code of the corresponding customer’s phone number
- `Phone`: the remaining seven-digit phone number
- `Int’l Plan`: whether the customer has an international calling plan: yes/no
- `VMail Plan`: whether the customer has a voice mail feature: yes/no
- `VMail Message`: presumably the average number of voice mail messages per month
- `Day Mins`: the total number of calling minutes used during the day
- `Day Calls`: the total number of calls placed during the day
- `Day Charge`: the billed cost of daytime calls
- `Eve Mins, Eve Calls, Eve Charge`: the billed cost for calls placed during the evening
- `Night Mins`, `Night Calls`, `Night Charge`: the billed cost for calls placed during nighttime
- `Intl Mins`, `Intl Calls`, `Intl Charge`: the billed cost for international calls
- `CustServ Calls`: the number of calls placed to Customer Service
- `Churn?`: whether the customer left the service: true/false

The last attribute, `Churn?`, is known as the target attribute–the attribute that we want the ML model to predict.  Because the target attribute is binary, our model will be performing binary prediction, also known as binary classification.

Let's begin exploring the data: